In [ ]:
! pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

In [ ]:
!sudo apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,223 kB]
Fetched 1,480 kB in 2s (929 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jam

In [ ]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libleptonica-dev is already the newest version (1.82.0-3build1).
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-script-latn is already the newest version (1:4.00~git30-7274cfa-1.1).
python3-pil is already the newest version (9.0.1-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!pip install unstructured-pytesseract
!pip install tesseract-ocr

In [ ]:
import os
from unstructured.partition.pdf import partition_pdf

In [ ]:
base_directory = "/content/dhvani_seller_db/ac37183260@gmail.com"

In [ ]:
file_paths = [os.path.join(base_directory, file) for file in os.listdir(base_directory) if file.endswith(".pdf")]

In [ ]:
# Initialize lists for storing extracted content
combined_results = []

In [ ]:
for idx, file_path in enumerate(file_paths):
    print(f"Processing file: {file_path}")

    # Dynamically naming variables for raw_pdf_elements
    raw_pdf_elements = partition_pdf(
        filename=file_path,
        strategy="hi_res",
        extract_images_in_pdf=True,
        extract_image_block_types=["Image", "Table"],
        extract_image_block_to_payload=False,
        extract_image_block_output_dir="extracted_data_seller"
    )

    # Extract unique rows and categorize elements
    NarrativeText = []
    title = []
    ListItem = []
    tab = []

    for element in raw_pdf_elements:
        element_type = str(type(element))
        if "unstructured.documents.elements.NarrativeText" in element_type:
            NarrativeText.append(str(element))
        elif "unstructured.documents.elements.Title" in element_type:
            title.append(str(element))
        elif "unstructured.documents.elements.ListItem" in element_type:
            ListItem.append(str(element))
        elif "unstructured.documents.elements.Table" in element_type:
            tab.append(str(element))

    # Combine extracted content
    combined = tab + ListItem + NarrativeText
    combined_results.append({
        "file_path": file_path,
        "NarrativeText": NarrativeText,
        "title": title,
        "ListItem": ListItem,
        "Table": tab,
        "combined": combined,
    })

Processing file: /content/dhvani_seller_db/ac37183260@gmail.com/bill.pdf
Processing file: /content/dhvani_seller_db/ac37183260@gmail.com/CHEMICALCONTENT.docx (1).pdf
Processing file: /content/dhvani_seller_db/ac37183260@gmail.com/Annex-3-3.1-Certificate-of-Origin (1).pdf


In [ ]:
combined_results

[{'file_path': '/content/dhvani_seller_db/ac37183260@gmail.com/bill.pdf',
  'NarrativeText': [],
  'title': ['SUBTOTAL: USD 2500', 'TOTAL: 2800'],
  'ListItem': [],
  'Table': ['PRICE AMOUNT QTY DESCRIPTION 5 BAGS PASHMINA SHAWL - USD 500 USD 2500 345NUM-678CHECK USD 2500'],
  'combined': ['PRICE AMOUNT QTY DESCRIPTION 5 BAGS PASHMINA SHAWL - USD 500 USD 2500 345NUM-678CHECK USD 2500']},
 {'file_path': '/content/dhvani_seller_db/ac37183260@gmail.com/CHEMICALCONTENT.docx (1).pdf',
  'NarrativeText': ['Issued by: TexLab International Testing ServicesAddress: B-32, Industrial Area, Phase 2, New Delhi, India - 110020Contact: +91-9812345678 | Email: info@texlabtesting.comCertificate No.: TEX-2024-PS-00123Date: 2024-11- 23'],
  'title': ['TEXLAB INTERNATIONAL',
   'TexLab International Testing Services',
   'Product Details',
   'Seller Details',
   'Compliance Statement',
   'e',
   'Phase 2, New Delhi, India -'],
  'ListItem': ['Product Name: Pashmina Shawl',
   'HS Code: 6214.20.10',
   '

In [ ]:
import json
import os
from typing import Dict, Any

class SellerPortfolioCreator:
    def __init__(self, base_path: str):
        self.base_path = base_path

    def extract_email(self, data: Dict[str, Any]) -> str:
        """Extract email from the seller information."""
        return data['directory_structure']['subdirectories']['seller_info']['company_details.json']['contact']['email']

    def create_directory_structure(self, email: str) -> str:
        """Create the main directory structure using the seller's email."""
        seller_path = os.path.join(self.base_path, email)
        os.makedirs(seller_path, exist_ok=True)

        # Create subdirectories
        for subdir in ['seller_info', 'product_info', 'lab_testing', 'shipping_docs', 'invoices']:
            os.makedirs(os.path.join(seller_path, subdir), exist_ok=True)

        return seller_path

    def write_json_file(self, path: str, data: Dict[str, Any]):
        """Write JSON data to a file with proper formatting."""
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

    def create_portfolio(self, portfolio_data: Dict[str, Any]):
        """Create the complete portfolio structure with all files."""
        # Extract email and create directory structure
        email = self.extract_email(portfolio_data)
        base_seller_path = self.create_directory_structure(email)

        # Get the subdirectories data
        subdirs = portfolio_data['directory_structure']['subdirectories']

        # Create each file in the appropriate subdirectory
        for subdir, files in subdirs.items():
            subdir_path = os.path.join(base_seller_path, subdir)
            for filename, content in files.items():
                file_path = os.path.join(subdir_path, filename)
                self.write_json_file(file_path, content)

        return base_seller_path

def main():
    # Your JSON data
    portfolio_data = {
        "directory_structure": {
            "root": "/seller_portfolio",
            "subdirectories": {
                "seller_info": {
                    "company_details.json": {
                        "company_name": "Royal Kashmir Exports",
                        "gst_number": "07ABCDE1234F1Z5",
                        "address": {
                            "street": "12, Silk Road",
                            "city": "Srinagar",
                            "state": "Jammu & Kashmir",
                            "postal_code": "190001",
                            "country": "India"
                        },
                        "contact": {
                            "phone": "+91-9876543210",
                            "email": "ac37183260@gmail.com"
                        }
                    }
                },
                "product_info": {
                    "product_details.json": {
                        "name": "Pashmina Shawl",
                        "hs_code": "6214.20.10",
                        "description": "100% Handwoven Pashmina Wool Shawl",
                        "material_composition": {
                            "pashmina_wool": "100%",
                            "other_fibers": "0%"
                        }
                    }
                },
                "lab_testing": {
                    "chemical_analysis.json": {
                        "certificate_details": {
                            "certificate_no": "TEX-2024-PS-00123",
                            "date": "2024-11-23",
                            "sample_id": "PSH-2024-1111",
                            "testing_date": "2024-11-20"
                        },
                        "testing_facility": {
                            "name": "TexLab International Testing Services",
                            "address": "B-32, Industrial Area, Phase 2, New Delhi, India - 110020",
                            "contact": {
                                "phone": "+91-9812345678",
                                "email": "info@texlabtesting.com"
                            },
                            "accreditation": "ISO/IEC 17025 Certified"
                        },
                        "test_results": {
                            "wool_protein_content": "98.7%",
                            "heavy_metal_content": "≤ 0.001 mg/kg",
                            "azo_dye_test": "Azo Dye-Free",
                            "formaldehyde_content": "0.002 mg/kg"
                        },
                        "certification": {
                            "authorized_by": {
                                "name": "Dr. Ramesh Chandra",
                                "designation": "Lab Manager"
                            },
                            "validity": "180 days from date of issuance"
                        }
                    }
                },
                "shipping_docs": {
                    "export_details.json": {
                        "consignor": "Rahul Kumar",
                        "consignor_address": "A/56 Vikas Colony, Main Market-18, Gurugram- 110038, India",
                        "departure_date": "2024-11-23",
                        "transport": {
                            "mode": "Air",
                            "flight_number": "AI-101",
                            "port_of_discharge": "Indira Gandhi International Airport, Delhi"
                        },
                        "shipment_details": {
                            "package_marks": "12345ABC",
                            "quantity": "5 bags",
                            "gross_weight": "75 kg",
                            "fob_value": "2000 USD"
                        }
                    }
                },
                "invoices": {
                    "sales_invoice.json": {
                        "invoice_details": {
                            "description": "Pashmina Shawl",
                            "quantity": "5 BAGS",
                            "unit_price": "500 USD",
                            "subtotal": "2500 USD",
                            "total": "2800 USD",
                            "reference_number": "345NUM-678CHECK"
                        }
                    }
                }
            }
        }
    }

    # Create portfolio in the specified location
    creator = SellerPortfolioCreator('/content/dhvani_seller_db')
    created_path = creator.create_portfolio(portfolio_data)
    print(f"Portfolio created successfully at: {created_path}")

if __name__ == "__main__":
    main()

Portfolio created successfully at: /content/dhvani_seller_db/ac37183260@gmail.com


In [ ]:
portfolio_data = {
        "directory_structure": {
            "root": "/seller_portfolio",
            "subdirectories": {
                "seller_info": {
                    "company_details.json": {
                        "company_name": "Royal Kashmir Exports",
                        "gst_number": "07ABCDE1234F1Z5",
                        "address": {
                            "street": "12, Silk Road",
                            "city": "Srinagar",
                            "state": "Jammu & Kashmir",
                            "postal_code": "190001",
                            "country": "India"
                        },
                        "contact": {
                            "phone": "+91-9876543210",
                            "email": "ac37183260@gmail.com"
                        }
                    }
                },
                "product_info": {
                    "product_details.json": {
                        "name": "Pashmina Shawl",
                        "hs_code": "6214.20.10",
                        "description": "100% Handwoven Pashmina Wool Shawl",
                        "material_composition": {
                            "pashmina_wool": "100%",
                            "other_fibers": "0%"
                        }
                    }
                },
                "lab_testing": {
                    "chemical_analysis.json": {
                        "certificate_details": {
                            "certificate_no": "TEX-2024-PS-00123",
                            "date": "2024-11-23",
                            "sample_id": "PSH-2024-1111",
                            "testing_date": "2024-11-20"
                        },
                        "testing_facility": {
                            "name": "TexLab International Testing Services",
                            "address": "B-32, Industrial Area, Phase 2, New Delhi, India - 110020",
                            "contact": {
                                "phone": "+91-9812345678",
                                "email": "info@texlabtesting.com"
                            },
                            "accreditation": "ISO/IEC 17025 Certified"
                        },
                        "test_results": {
                            "wool_protein_content": "98.7%",
                            "heavy_metal_content": "≤ 0.001 mg/kg",
                            "azo_dye_test": "Azo Dye-Free",
                            "formaldehyde_content": "0.002 mg/kg"
                        },
                        "certification": {
                            "authorized_by": {
                                "name": "Dr. Ramesh Chandra",
                                "designation": "Lab Manager"
                            },
                            "validity": "180 days from date of issuance"
                        }
                    }
                },
                "shipping_docs": {
                    "export_details.json": {
                        "consignor": "Rahul Kumar",
                        "consignor_address": "A/56 Vikas Colony, Main Market-18, Gurugram- 110038, India",
                        "departure_date": "2024-11-23",
                        "transport": {
                            "mode": "Air",
                            "flight_number": "AI-101",
                            "port_of_discharge": "Indira Gandhi International Airport, Delhi"
                        },
                        "shipment_details": {
                            "package_marks": "12345ABC",
                            "quantity": "5 bags",
                            "gross_weight": "75 kg",
                            "fob_value": "2000 USD"
                        }
                    }
                },
                "invoices": {
                    "sales_invoice.json": {
                        "invoice_details": {
                            "description": "Pashmina Shawl",
                            "quantity": "5 BAGS",
                            "unit_price": "500 USD",
                            "subtotal": "2500 USD",
                            "total": "2800 USD",
                            "reference_number": "345NUM-678CHECK"
                        }
                    }
                }
            }
        }
    }

In [ ]:
!pip install together


  Using cached together-1.3.5-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0


In [ ]:
import os
from together import Together


In [ ]:
api_key = "0fcc4fc5facac8d43a0c34fac13836223ba7c7033f80767ea4670b6be2c4e088"  # Replace with your actual API key
client = Together(api_key=api_key)

In [ ]:
# Prepare the combined input
user_message = '''You are an advanced AI assistant. Here is the task:
- Extract all official information such as documents of the seller, buyer, lab testing results, product details, taxes, etc., from the given data.
- Organize the information into a JSON format that can be used for future document generation as a user portfolio.
- If there is sufficient data, create a directory and add multiple files to it for modular data handling. Return the directory and file paths as output in this case.
'''

# Send data to the Together model
response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[
        {"role": "system", "content": user_message},
        {"role": "user", "content": f"Data: {combined_results}"}
    ]
)

# Display the output from the model
try:
    result = response.choices[0].message.content
except AttributeError as e:
    print(f"Error: {e}")

In [ ]:
result

'I extracted the official information from the given data and organized it into a JSON format. Since there is sufficient data, I created a directory and added multiple files to it for modular data handling. Here is the output:\n\n**Directory:** `seller_portfolio`\n\n**Files:**\n\n1. `seller_info.json`: Contains information about the seller, including name, email, phone number, and GST number.\n```json\n{\n  "seller_name": "Royal Kashmir Exports",\n  "email": "ac37183260@gmail.com",\n  "phone_number": "+91-9876543210",\n  "gst_number": "07ABCDE1234F1Z5"\n}\n```\n2. `product_info.json`: Contains information about the product, including name, description, HS code, and material composition.\n```json\n{\n  "product_name": "Pashmina Shawl",\n  "description": "100% Handwoven Pashmina Wool Shawl",\n  "hs_code": "6214.20.10",\n  "material_composition": "Pashmina Wool: 100%, Other Fibers: 0%"\n}\n```\n3. `lab_test_results.json`: Contains information about the lab test results, including chemical

In [ ]:
user_message = '''You are an advanced AI assistant. Here is the task:
- convert this into a json file result variable, which is a Python dictionary containing the structured data from the information you provided. The variable holds key-value pairs where each key represents a filename (e.g., seller_info.json, product_info.json, etc.) and each value is a dictionary containing relevant data in JSON format.

Here is the breakdown of the result variable:
result = {
    "seller_info.json": { ... },
    "product_info.json": { ... },
    "lab_test_results.json": { ... },
    "certificate_of_origin.json": { ... },
    "tax_info.json": { ... }
}
'''

# Send data to the Together model
response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[
        {"role": "system", "content": user_message},
        {"role": "user", "content": f"Data: {result}"}
    ]
)

# Display the output from the model
try:
    result_final = response.choices[0].message.content
except AttributeError as e:
    print(f"Error: {e}")

In [ ]:
result_final

'Here is the Python code to convert the provided data into a JSON file result variable:\n```python\nimport json\n\nresult = {\n    \'seller_info.json\': {\n        \'file_path\': \'/content/dhvani_seller_db/ac37183260@gmail.com/bill.pdf\',\n        \'NarrativeText\': [],\n        \'title\': [\'SUBTOTAL: USD 2500\', \'TOTAL: 2800\'],\n        \'ListItem\': [],\n        \'Table\': [\'PRICE AMOUNT QTY DESCRIPTION 5 BAGS PASHMINA SHAWL - USD 500 USD 2500 345NUM-678CHECK USD 2500\'],\n        \'combined\': [\'PRICE AMOUNT QTY DESCRIPTION 5 BAGS PASHMINA SHAWL - USD 500 USD 2500 345NUM-678CHECK USD 2500\']\n    },\n    \'product_info.json\': {\n        \'file_path\': \'/content/dhvani_seller_db/ac37183260@gmail.com/CHEMICALCONTENT.docx (1).pdf\',\n        \'NarrativeText\': [\'Issued by: TexLab International Testing ServicesAddress: B-32, Industrial Area, Phase 2, New Delhi, India - 110020Contact: +91-9812345678 | Email: info@texlabtesting.comCertificate No.: TEX-2024-PS-00123Date: 2024-11- 

In [ ]:
result2 = {
    "seller_info.json": {
        "seller_name": "Royal Kashmir Exports",
        "email": "ac37183260@gmail.com",
        "phone_number": "+91-9876543210",
        "gst_number": "07ABCDE1234F1Z5"
    },
    "product_info.json": {
        "product_name": "Pashmina Shawl",
        "description": "100% Handwoven Pashmina Wool Shawl",
        "hs_code": "6214.20.10",
        "material_composition": "Pashmina Wool: 100%, Other Fibers: 0%"
    },
    "lab_test_results.json": {
        "chemical_content_analysis": "Wool Protein Content: 98.7%, Heavy Metal Content: Below detectable limits (≤ 0.001 mg/kg)"
    },
    "certificate_of_origin.json": {
        "country_of_origin": "India",
        "description_of_goods": "5 bags of textile-pashmina shawls (woolen fabric, HS: 5208)",
        "harmonized_system_number": "5208"
    },
    "tax_info.json": {
        "subtotal": 2500,
        "total": 2800
    }
}

In [ ]:
import os
import json


# Extract seller email to create a unique directory
seller_email = result["seller_info.json"]["email"]
directory_path = f"/content/dhvani_seller_db/{seller_email}/bin"

# Create the directory if it doesn't exist
os.makedirs(directory_path, exist_ok=True)

# Iterate over the result dictionary and save each file
for file_name, file_content in result.items():
    file_path = os.path.join(directory_path, file_name)
    with open(file_path, "w") as file:
        json.dump(file_content, file, indent=4)

print(f"Data has been saved to: {directory_path}")


Data has been saved to: /content/dhvani_seller_db/ac37183260@gmail.com/bin


In [ ]:
user_message = '''You are an advanced AI assistant. Here is the task:
- Search through all the subdirectories of the directory "/content/dhvani_seller_db/ac37183260@gmail.com" for any `.json` files.
- Extract the relevant data from these files and generate a structured packing list document based on the data.
- The document should follow the template provided in "/content/dhvani_seller_db/templates_documents.json".
- Use the template data to format the packing list document, including any dynamic fields such as the title and other sections.
- Once the packing list is generated, return the file path to the resulting document.

Here is the breakdown of the expected actions:
1. Traverse through all subdirectories to find `.json` files.
2. Load the template from "/content/dhvani_seller_db/templates_documents.json".
3. Create a Word document that includes all relevant packing data extracted from the `.json` files.
4. Return the path to the generated Word document.

Please proceed with the task and return the final document location.'''

# Send data to the Together model
response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[
        {"role": "system", "content": user_message},
        {"role": "user", "content": "Task to generate packing list document based on data from JSON files and template"}
    ]
)

# Display the output from the model
try:
    result_final = response.choices[0].message.content
except AttributeError as e:
    print(f"Error: {e}")


In [ ]:
result_final

'I\'ll proceed with the task. Please note that I\'ll assume the directory structure and file names are correct.\n\n**Step 1: Traverse through all subdirectories to find .json files**\n\nI\'ll use the `os` module to traverse through all subdirectories of the specified directory and find all `.json` files.\n\n```\nimport os\nimport json\nfrom docx import Document\n\n# Specify the directory path\ndirectory_path = \'/content/dhvani_seller_db/ac37183260@gmail.com\'\n\n# Initialize an empty list to store the JSON files\njson_files = []\n\n# Traverse through all subdirectories\nfor root, dirs, files in os.walk(directory_path):\n    for file in files:\n        if file.endswith(\'.json\'):\n            json_files.append(os.path.join(root, file))\n\n# Load the template from the specified file\nwith open(\'/content/dhvani_seller_db/templates_documents.json\') as f:\n    template_data = json.load(f)\n\n# Create a Word document\ndocument = Document()\n\n# Set the title and other sections based on t

In [ ]:
import os
import json
from docx import Document

# Function to search for all .json files in a given directory and subdirectories
def find_json_files(directory):
    json_files = []
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(subdir, file))
    return json_files

# Function to load template from the templates_documents.json file
def load_template(template_file):
    with open(template_file, 'r') as f:
        template_data = json.load(f)
    return template_data


In [ ]:
edoc= 'packing list'

In [ ]:
import os
import json
from docx import Document

# Function to search for all .json files in a given directory and subdirectories
def find_json_files(directory):
    json_files = []
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(subdir, file))
    return json_files

# Function to load template from the templates_documents.json file
def load_template(template_file):
    with open(template_file, 'r') as f:
        template_data = json.load(f)
    return template_data

# Base directory for searching json files
base_directory = "/content/dhvani_seller_db/ac37183260@gmail.com"

# Search for all .json files in the base directory
json_files = find_json_files(base_directory)

# Load the template from the template file
template_file = "/content/dhvani_seller_db/templates_documents.json"
template_data = load_template(template_file)

# Extract data from the JSON files and convert it into a string
json_contents = ""
for json_file in json_files:
    with open(json_file, 'r') as f:
        json_contents += json.dumps(json.load(f), indent=2) + "\n\n"


In [ ]:
template_data

{'documents': [{'name': 'Proforma Invoice',
   'template': {'Exporter Details': '<Exporter Name, Address, Contact Details>',
    'Invoice Number': '<Invoice Number>',
    'Date': '<Date>',
    'Buyer Details': '<Buyer Name, Address, Contact Details>',
    'Description of Goods': '<Product Description>',
    'Quantity': '<Quantity>',
    'Unit Price': '<Unit Price>',
    'Total Price': '<Total Price>',
    'Terms of Payment': '<Terms of Payment>',
    'Terms of Delivery': '<Terms of Delivery>',
    'Signature': '<Authorized Signature>'}},
  {'name': 'Commercial Invoice',
   'template': {'Exporter Details': '<Exporter Name, Address, Contact Details>',
    'Invoice Number': '<Invoice Number>',
    'Date': '<Date>',
    'Buyer Details': '<Buyer Name, Address, Contact Details>',
    'Consignee': '<Consignee Name, Address>',
    'Shipment Details': {'Mode of Transport': '<Mode of Transport>',
     'Port of Loading': '<Port of Loading>',
     'Port of Discharge': '<Port of Discharge>'},
    '

In [ ]:
portfolio_data

{'directory_structure': {'root': '/seller_portfolio',
  'subdirectories': {'seller_info': {'company_details.json': {'company_name': 'Royal Kashmir Exports',
     'gst_number': '07ABCDE1234F1Z5',
     'address': {'street': '12, Silk Road',
      'city': 'Srinagar',
      'state': 'Jammu & Kashmir',
      'postal_code': '190001',
      'country': 'India'},
     'contact': {'phone': '+91-9876543210', 'email': 'ac37183260@gmail.com'}}},
   'product_info': {'product_details.json': {'name': 'Pashmina Shawl',
     'hs_code': '6214.20.10',
     'description': '100% Handwoven Pashmina Wool Shawl',
     'material_composition': {'pashmina_wool': '100%', 'other_fibers': '0%'}}},
   'lab_testing': {'chemical_analysis.json': {'certificate_details': {'certificate_no': 'TEX-2024-PS-00123',
      'date': '2024-11-23',
      'sample_id': 'PSH-2024-1111',
      'testing_date': '2024-11-20'},
     'testing_facility': {'name': 'TexLab International Testing Services',
      'address': 'B-32, Industrial Area

In [ ]:
user_message = f'''
You are an advanced AI assistant. Here is the task:
- Extract the relevant data and create a packing list document required for exporting the items to usa from india
 Create a Word document that includes all relevant packing data extracted from user information in the result
 it must be well formatted and can be used for official purposes.

 expected result - only the code of the document so that it can be converted into .docx file using python libraries



Please proceed with the task and return the final document.

Template Data: {template_data}
Result: {portfolio_data}

return the complete document with all the details filled and the complete document with all the seller details or product details etc
'''

# Send data to the Together model
response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[
        {"role": "system", "content": user_message},
        {"role": "user", "content": f"Data: {result}" }
    ]
)

# Display the output from the model
try:
    result_final = response.choices[0].message.content
    print(result_final)
except AttributeError as e:
    print(f"Error: {e}")

Here is the packing list document with all the relevant data filled:

**Packing List**

**Exporter Details**

* Exporter Name: Royal Kashmir Exports
* Address: 12, Silk Road, Srinagar, Jammu & Kashmir, India - 190001
* Contact Details: Phone: +91-9876543210, Email: ac37183260@gmail.com

**Packing List Number:** PL-001
**Date:** 2024-11-23

**Buyer Details**

* Buyer Name: [Insert Buyer Name]
* Address: [Insert Buyer Address]

**Shipment Details**

* Mode of Transport: Air
* Port of Loading: Indira Gandhi International Airport, Delhi
* Port of Discharge: [Insert Port of Discharge]

**Package Details**

| Package Number | Description of Contents | Weight (kg) | Dimensions (L x W x H) |
| --- | --- | --- | --- |
| 1 | 5 bags of Pashmina Shawls | 75 | 100 x 50 x 20 |

**Total Packages:** 1

**Signature:** [Insert Authorized Signature]

**Note:** The above packing list is a sample and may need to be modified according to the specific requirements of the shipment.

Here is the code for the d

In [ ]:
import json
from docx import Document

data = {
    'seller_info.json': {
        'seller_name': 'Royal Kashmir Exports',
        'email': 'ac37183260@gmail.com',
        'phone_number': '+91-9876543210',
        'gst_number': '07ABCDE1234F1Z5',
        'address': 'Kashmir, India'
    },
    'product_info.json': {
        'product_name': 'Pashmina Shawl',
        'description': '100% Handwoven Pashmina Wool Shawl',
        'hs_code': '6214.20.10',
        'material_composition': 'Pashmina Wool: 100%, Other Fibers: 0%'
    },
    'lab_test_results.json': {
        'chemical_content_analysis': 'Wool Protein Content: 98.7%, Heavy Metal Content: Below detectable limits (≤ 0.001 mg/kg)'
    },
    'certificate_of_origin.json': {
        'country_of_origin': 'India',
        'description_of_goods': '5 bags of textile-pashmina shawls (woolen fabric, HS: 5208)',
        'harmonized_system_number': '5208'
    },
    'tax_info.json': {
        'subtotal': 2500,
        'total': 2800
    },
    'packing_list_number': 'PL12345',
    'date': '2024-11-23',
    'shipment_details': {
        'mode_of_transport': 'Air',
        'port_of_loading': 'Delhi International Airport'
    },
    'package_details': [
        {'package_number': '1', 'description_of_contents': 'Pashmina Shawl', 'weight': 5},
        {'package_number': '2', 'description_of_contents': 'Pashmina Shawl', 'weight': 5}
    ],
    'total_packages': 2,
    'total_weight': 10
}

document = Document()

# Exporter Details
document.add_paragraph("Exporter Details")
document.add_paragraph(f"Exporter Name: {data['seller_info.json']['seller_name']}")
document.add_paragraph(f"Address: {data['seller_info.json']['address']}")
document.add_paragraph(f"Contact Details: Phone: {data['seller_info.json']['phone_number']}, Email: {data['seller_info.json']['email']}")

# Packing List Number and Date
document.add_paragraph("Packing List Number:")
document.add_paragraph(f"{data['packing_list_number']}")
document.add_paragraph("Date:")
document.add_paragraph(f"{data['date']}")

# Buyer Details
document.add_paragraph("Buyer Details")
document.add_paragraph(f"Buyer Name: [Insert Buyer Name]")
document.add_paragraph(f"Address: [Insert Buyer Address]")

# Shipment Details
document.add_paragraph("Shipment Details")
document.add_paragraph(f"Mode of Transport: {data['shipment_details']['mode_of_transport']}")
document.add_paragraph(f"Port of Loading: {data['shipment_details']['port_of_loading']}")
document.add_paragraph(f"Port of Discharge: [Insert Port of Discharge]")

# Package Details
document.add_paragraph("Package Details")
for package in data['package_details']:
    document.add_paragraph(f"Package Number: {package['package_number']}")
    document.add_paragraph(f"Description of Contents: {package['description_of_contents']}")
    document.add_paragraph(f"Weight (kg): {package['weight']}")

# Total Packages and Weight
document.add_paragraph(f"Total Packages: {data['total_packages']}")
document.add_paragraph(f"Total Weight: {data['total_weight']} kg")

# Signature
document.add_paragraph("Signature:")
document.add_paragraph(f"[Authorized Signature]")

document.save("packing_list.docx")
